In [7]:
import numpy as np
import cv2 as cv

In [17]:
def G_x(x,sigma):
    import math
    return math.exp(-(x**2)/(2*(sigma**2)))/(sigma*((2*math.pi)**0.5))


In [18]:
def kernel_calculation(part,sigma_r,sigma_s):
    sum = 0.0
    normalization = 0.0
    pi = part.shape[0]//2
    for i in range(part.shape[0]):
        for j in range(part.shape[1]):
            distance = ((i-pi)**2 + (j-pi)**2)**0.5
            intensity_difference = abs(part[i][j] - part[pi][pi])
            normalization+=(G_x(distance,sigma_s)*G_x(intensity_difference,sigma_r))
            sum += G_x(distance,sigma_s)*G_x(intensity_difference,sigma_r)*part[i,j]
    return sum/normalization

In [19]:
def bilateral_filter(image,kernel_size,sigma_s,sigma_r):
    border = kernel_size//2
    img = image.copy()
    return_img = np.zeros(img.shape)
    img = cv.copyMakeBorder(img, border, border, border,border, cv.BORDER_REPLICATE, None)

    for x in range(return_img.shape[0]):
        for y in range(return_img.shape[1]):
            part = img[x:x+2*border+1, y:y+2*border+1]
            return_img[x,y] = kernel_calculation(part,sigma_s,sigma_r)
    
    return return_img

In [52]:
noisyImage_Gaussian_01 = cv.imread("noisyImage_Gaussian_01.jpg", cv.IMREAD_GRAYSCALE)
normalized_noisyImage_Gaussian_01 = np.single(noisyImage_Gaussian_01)
normalized_noisyImage_Gaussian_01 = cv.normalize(normalized_noisyImage_Gaussian_01, None, alpha=0,beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

noisyImage_Gaussian = cv.imread("noisyImage_Gaussian.jpg", cv.IMREAD_GRAYSCALE)
normalized_noisyImage_Gaussian = np.single(noisyImage_Gaussian)
normalized_noisyImage_Gaussian = cv.normalize(normalized_noisyImage_Gaussian, None, alpha=0,beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

In [53]:
cv1 = cv.bilateralFilter(normalized_noisyImage_Gaussian_01, 5, 0.1, 1, borderType = cv.BORDER_REPLICATE)
cv1 = cv.normalize(cv1, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

cv2 = cv.bilateralFilter(normalized_noisyImage_Gaussian, 5, 3, 0.9, borderType = cv.BORDER_REPLICATE)
cv2 = cv.normalize(cv2, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

In [54]:
mine1 = bilateral_filter(normalized_noisyImage_Gaussian_01,5, 0.1, 1)
mine1 = cv.normalize(mine1, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

mine2 = bilateral_filter(normalized_noisyImage_Gaussian,5, 3, 0.9)
mine2 = cv.normalize(mine2, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

In [55]:
cv.imshow("Bilateral filtered noisyImage_Gaussian_01 output(OPENCV)",mine1.astype(np.uint8))

cv.imshow("Bilateral filtered noisyImage_Gaussian_01 output(My Filter)",mine1.astype(np.uint8))

cv.imshow("Bilateral filtered noisyImage_Gaussian output(OPENCV)",mine1.astype(np.uint8))

cv.imshow("Bilateral filtered noisyImage_Gaussian output(My Filter)",mine1.astype(np.uint8))

cv.waitKey(0)
cv.destroyAllWindows()

In [64]:
abs_img1 = abs(mine1.astype(float)-cv1.astype(float))
print("[noisyImage_Gaussian_01] Max pixel intensity difference:",round(abs_img1.max()))

[noisyImage_Gaussian_01] Max pixel intensity difference: 3


In [65]:
abs_img2 = abs(mine2.astype(float)-cv2.astype(float))
print("[noisyImage_Gaussian] Max pixel intensity difference:",round(abs_img2.max()))

[noisyImage_Gaussian] Max pixel intensity difference: 8
